In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tools_for_nlp').getOrCreate()

22/05/26 13:43:47 WARN Utils: Your hostname, ls-Aspire-A515-51G resolves to a loopback address: 127.0.1.1; using 192.168.15.8 instead (on interface wlp3s0)
22/05/26 13:43:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/26 13:43:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Tools for NLP - Part 01

In [2]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer

In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [13]:
sen_df = spark.createDataFrame([
    (0, 'I heard about spark'),
    (1, 'I wish java could use case classes'),
    (2, 'Logistic,regression,models,are,neat')
], ['id', 'sentence'])

In [14]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0| I heard about spark|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [7]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [9]:
regex_tokenizer = RegexTokenizer(inputCol='sentence', outputCol='words',
                                pattern='\\W')

In [10]:
count_tokens = udf(lambda words: len(words), IntegerType())

In [15]:
tokenized = tokenizer.transform(sen_df)

In [16]:
tokenized.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0| I heard about spark|[i, heard, about,...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [17]:
tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0| I heard about spark|[i, heard, about,...|     4|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [18]:
rg_tokenized = regex_tokenizer.transform(sen_df)

In [19]:
rg_tokenized.withColumn('tokens', count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0| I heard about spark|[i, heard, about,...|     4|
|  1|I wish java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [20]:
from pyspark.ml.feature import StopWordsRemover

In [22]:
sentenceDataFrame = spark.createDataFrame([
    (0, ['I', 'saw', 'the', 'green', 'horse']),
    (1, ['Mary', 'had', 'a', 'little', 'lamb'])
], ['id', 'tokens'])

In [23]:
sentenceDataFrame.show()

+---+--------------------+
| id|              tokens|
+---+--------------------+
|  0|[I, saw, the, gre...|
|  1|[Mary, had, a, li...|
+---+--------------------+



In [24]:
remover = StopWordsRemover(inputCol='tokens', outputCol='filtered')

In [25]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [26]:
# n-gram
from pyspark.ml.feature import NGram

In [31]:
wordDataFrame = spark.createDataFrame([
    (0, ['I' , 'heard', 'about' , 'spark']),
    (1, ['I', 'wish', 'java', 'could', 'use', 'case', 'classes']),
    (2, ['Logistic', 'regression', 'models', 'are', 'neat'])
], ['id', 'words'])

In [32]:
wordDataFrame.show()

+---+--------------------+
| id|               words|
+---+--------------------+
|  0|[I, heard, about,...|
|  1|[I, wish, java, c...|
|  2|[Logistic, regres...|
+---+--------------------+



In [33]:
ngram = NGram(n=2, inputCol='words', outputCol='grams') # Strings in consecutive words

In [35]:
ngram.transform(wordDataFrame).show(truncate=False)

+---+------------------------------------------+------------------------------------------------------------------+
|id |words                                     |grams                                                             |
+---+------------------------------------------+------------------------------------------------------------------+
|0  |[I, heard, about, spark]                  |[I heard, heard about, about spark]                               |
|1  |[I, wish, java, could, use, case, classes]|[I wish, wish java, java could, could use, use case, case classes]|
|2  |[Logistic, regression, models, are, neat] |[Logistic regression, regression models, models are, are neat]    |
+---+------------------------------------------+------------------------------------------------------------------+



## Tools for NLP - Part 02

In [37]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [41]:
sen_df.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0| I heard about spark|
|  1|I wish java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [42]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [43]:
words_data = tokenizer.transform(sen_df)

In [44]:
words_data.show()

+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0| I heard about spark|[i, heard, about,...|
|  1|I wish java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



In [45]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [46]:
featurized_data = hashing_tf.transform(words_data)

In [52]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [53]:
idf_model = idf.fit(featurized_data)

In [54]:
rescaled_data = idf_model.transform(featurized_data)

In [55]:
rescaled_data.printSchema()

root
 |-- id: long (nullable = true)
 |-- sentence: string (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- rawFeatures: vector (nullable = true)
 |-- features: vector (nullable = true)



In [56]:
rescaled_data.show()

+---+--------------------+--------------------+--------------------+--------------------+
| id|            sentence|               words|         rawFeatures|            features|
+---+--------------------+--------------------+--------------------+--------------------+
|  0| I heard about spark|[i, heard, about,...|(262144,[18700,19...|(262144,[18700,19...|
|  1|I wish java could...|[i, wish, java, c...|(262144,[19036,20...|(262144,[19036,20...|
|  2|Logistic,regressi...|[logistic,regress...|(262144,[11534],[...|(262144,[11534],[...|
+---+--------------------+--------------------+--------------------+--------------------+



22/05/26 14:45:18 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
22/05/26 14:45:18 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [59]:
rescaled_data.select('features').show()

+--------------------+
|            features|
+--------------------+
|(262144,[18700,19...|
|(262144,[19036,20...|
|(262144,[11534],[...|
+--------------------+



22/05/26 14:46:40 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
22/05/26 14:46:40 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [60]:
from pyspark.ml.feature import CountVectorizer

In [62]:
df = spark.createDataFrame([
    (0, "a b c".split(" ")), 
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [63]:
df.show()

+---+---------------+
| id|          words|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [66]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [67]:
model = cv.fit(df)

In [68]:
results = model.transform(df)

In [69]:
results.show(truncate=False)

+---+---------------+-------------------------+
|id |words          |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

